#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [1]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /home/abhilash/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [2]:
from nltk.corpus import brown
brown.words()

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [6]:
from nltk.corpus import brown
import string

# Remove stop words
from nltk.corpus import stopwords
stop_words = stopwords.words(fileids='english')
listWords = brown.words()[:200]
brown_noStopWords = [w for w in listWords if w.lower() not in stop_words]

# Remove punctuation
import re
brown_noPunctuation = [re.sub(r'[^\w]','',w) for w in brown_noStopWords]
# Remove the leftover empty strings, so that they don't get counted in the
# window operation
brown_noPunctuation = [w for w in brown_noPunctuation if w != ''] 

# stem the words
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
brown_preprocessed = [ stemmer.stem(w) for w in brown_noPunctuation ]
print(brown_preprocessed[:50])

[u'fulton', u'counti', u'grand', u'juri', u'said', u'friday', u'investig', u'atlanta', u'recent', u'primari', u'elect', u'produc', u'evid', u'irregular', u'took', u'place', u'juri', u'said', u'termend', u'present', u'citi', u'execut', u'committe', u'overal', u'charg', u'elect', u'deserv', u'prais', u'thank', u'citi', u'atlanta', u'manner', u'elect', u'conduct', u'septemberoctob', u'term', u'juri', u'charg', u'fulton', u'superior', u'court', u'judg', u'durwood', u'pye', u'investig', u'report', u'possibl', u'irregular', u'hardfought', u'primari']


In [7]:
# The 5000 most common words are the vocabulary and the 1000 most common words
# are the Context.
dictWordCnt = {}
for token in brown_preprocessed:
    if token in dictWordCnt.keys():
        dictWordCnt[token] += 1
    else:
        dictWordCnt[token] = 1
# sort the dictionary
cnt = 0
listSortedKeys = sorted(dictWordCnt, key=dictWordCnt.get, reverse=True)
for w in listSortedKeys:
    cnt += 1
    print (w, ':', dictWordCnt[w])
    if cnt >= 20:
        break

(u'juri', ':', 6)
(u'elect', ':', 5)
(u'said', ':', 4)
(u'citi', ':', 3)
(u'fulton', ':', 3)
(u'number', ':', 2)
(u'investig', ':', 2)
(u'primari', ':', 2)
(u'charg', ':', 2)
(u'grand', ':', 2)
(u'report', ':', 2)
(u'atlanta', ':', 2)
(u'law', ':', 2)
(u'irregular', ':', 2)
(u'counti', ':', 1)
(u'recommend', ':', 1)
(u'often', ':', 1)
(u'execut', ':', 1)
(u'among', ':', 1)
(u'registr', ':', 1)


In [8]:
dictVocab2idx = {}
dictContext2idx = {}
dictIdx2Vocab = {}
dictIdx2Context = {}
for idx,token in enumerate(listSortedKeys[:5000]):
    dictVocab2idx[token] = idx
    dictIdx2Vocab[idx] = token
    if idx < 1000:
        dictIdx2Context[idx] = token
        dictContext2idx[token] = idx

cnt = 0
for key,val in dictIdx2Vocab.items():
    print(key,val)
    cnt += 1
    if cnt > 20:
        break

(0, u'juri')
(1, u'elect')
(2, u'said')
(3, u'citi')
(4, u'fulton')
(5, u'number')
(6, u'investig')
(7, u'primari')
(8, u'charg')
(9, u'grand')
(10, u'report')
(11, u'atlanta')
(12, u'law')
(13, u'irregular')
(14, u'counti')
(15, u'recommend')
(16, u'often')
(17, u'execut')
(18, u'among')
(19, u'registr')
(20, u'topic')


## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [ ]:
# Your Code Here...

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [ ]:
# Your Code Here...

## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [ ]:
# Your Code Here...

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [ ]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# Your Code Here...

## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [ ]:
# Your Code Here....

## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [ ]:
# Mean Absolute Error

In [ ]:
# Root Mean Squared Error

## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

In [ ]:
# Your Code Here...

*Insert discussion here*